## Airline Price Optimization Problem

### Problem Statement: 
FlyIndia is a fictional airlines company. FlyIndia would have two types of customer segments to cater to the early birds, i.e., the customers who would buy ticket much in advance and hence would be eligible for discount, and the late buyers or the ones who would be buying tickets at a regular prices. The airline company XYZ wants to maximize its revenue for a particular Delhi to Bangalore flight by allocating the regular and discounted seats judiciously. There are 166 seats available in the aircraft. Based on the past observations, regular demand and the discounted demand do not exceed 100 and 150 tickets respectively.

<b><i>The prices of the tickets are as follows:</i></b>
- Discounted : INR 1190
- Regular : INR 3085

In [1]:
from pyomo.environ import *

In [2]:
tkt_type = ['eb', 'reg']

## model.tkt_type = Set (initialize('eb', 'reg'))

In [3]:
tkt_price = {'eb':1190, 'reg':3085}

#model.tkt_price = Param(Set(initialize('eb', 'reg') initialize = tkt_price))

tkt_demand ={'eb':100, 'reg':150}

#model.tkt_demand = Param(Set(initialize('eb', 'reg'), initialize=tkt_demand))

In [4]:
model = ConcreteModel()

In [5]:
model.tkt_demand = Var(tkt_type, within=PositiveIntegers)

In [6]:
# objective

model.value = Objective(expr = sum(model.tkt_demand[i]*tkt_price[i] for i in tkt_type), sense=maximize)

In [7]:
### Constraints

### Regular Demand should not exceed 100
model.reg_demand = Constraint(expr = model.tkt_demand['reg'] <= 100)

### Discounted demand should not exceed 150
model.dis_demand = Constraint(expr = model.tkt_demand['eb'] <= 150)

### Supply Rule

def supply_constraint(model):
    return(sum(model.tkt_demand[i] for i in tkt_type) == 166)

model.supply = Constraint(rule = supply_constraint)

In [8]:
result = SolverFactory('glpk').solve(model)
result

{'Problem': [{'Name': 'unknown', 'Lower bound': 387040.0, 'Upper bound': 387040.0, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 3, 'Number of nonzeros': 5, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.08901023864746094}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [9]:
model.pprint()

1 Set Declarations
    tkt_demand_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'eb', 'reg'}

1 Var Declarations
    tkt_demand : Size=2, Index=tkt_demand_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         eb :     1 :  66.0 :  None : False : False : PositiveIntegers
        reg :     1 : 100.0 :  None : False : False : PositiveIntegers

1 Objective Declarations
    value : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 1190*tkt_demand[eb] + 3085*tkt_demand[reg]

3 Constraint Declarations
    dis_demand : Size=1, Index=None, Active=True
        Key  : Lower : Body           : Upper : Active
        None :  -Inf : tkt_demand[eb] : 150.0 :   True
    reg_demand : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :  -Inf : tkt_demand[reg] : 100.0 :   True
    supp

In [10]:
model.value()

387040.0